# Construcción de Modelos N-Gram con Suavizado Laplace


### 1. Importar Librerías Necesarias



In [2]:
import pandas as pd
from collections import defaultdict, Counter
import pickle


### 2. Cargar los Datasets

Aquí cargamos los archivos de entrenamiento de los datasets 20N y BAC. Los archivos están en formato CSV y contienen las oraciones preprocesadas en una columna llamada `unique`. Después de cargar los datos, verificamos el contenido con un `print` de las primeras filas.


In [3]:
datos20N = pd.read_csv('20N_01_training.csv')
datosBAC = pd.read_csv('BAC_01_training.csv')

print("Primeras filas del dataset 20N:")
print(datos20N.head())
print("\nPrimeras filas del dataset BAC:")
print(datosBAC.head())

Primeras filas del dataset 20N:
   Unnamed: 0                                             unique
0        3599  <s> on fri NUM apr NUM NUM NUM NUM gmt markfri...
1       12086  <s> can somebody please help me with informati...
2        4281  <s> hi there could some kind soul tell me what...
3       12513  <s> d9bertil dtek chalmers se bertil jonell wr...
4        7177  <s> in article 1993apr20 NUM NUM chpc org rbou...

Primeras filas del dataset BAC:
   Unnamed: 0                                             unique
0      260662  <s> sorry i havnt writen in so long i ve been ...
1      163528  <s> this is how i was feeling earlier today oh...
2      140457  <s> sometime ago i read this in a book the har...
3      288965  <s> to aj for listening to me tonight we cover...
4      330387  <s> there is something else i would like to bi...


### 3. Separación de Oraciones

Cada oración en los datasets ya está  preprocesada y almacenada en la columna `unique`. En esta sección, extraemos las oraciones y las almacenamos como listas de tokens para facilitar la construcción de los N-gramas.


In [4]:
def separarOraciones(datos):
    """
    Tokeniza las oraciones de un DataFrame a partir de la columna 'unique'

    Parámetros:
    datos (pd.DataFrame): DataFrame que contiene una columna 'unique' con las oraciones

    Retorna:
    list: Lista de oraciones tokenizadas, donde cada oración es una lista de tokens
    """
    oracionesSeparadas = []
    for oracion in datos['unique']:
        tokens = oracion.split() 
        oracionesSeparadas.append(tokens)
    return oracionesSeparadas

oracionesSeparadas20N = separarOraciones(datos20N)
oracionesSeparadasBAC = separarOraciones(datosBAC)

# Mostrar ejemplos
print("Ejemplos de oraciones tokenizadas del dataset 20N:")
print(oracionesSeparadas20N[:2])
print("\nEjemplos de oraciones tokenizadas del dataset BAC:")
print(oracionesSeparadasBAC[:2])


Ejemplos de oraciones tokenizadas del dataset 20N:
[['<s>', 'on', 'fri', 'NUM', 'apr', 'NUM', 'NUM', 'NUM', 'NUM', 'gmt', 'markfried', 'fellensiek', 'ins413j', 'mdw056', 'cc', 'monash', 'edu', 'au', 'wrote', 'if', 'you', 're', 'considering', 'buying', 'a', 'system', 'with', 'a', 'view', 'to', 'using', 'it', 'to', 'run', 'unix', 'linux', 'bsd', 'etc', 'or', 'some', 'other', 'special', 'software', 'there', 'is', 'a', 'good', 'chance', 'that', 'it', 'will', 'not', 'work', 'with', 'the', 'diamond', 'cards', 'this', 'is', 'due', 'to', 'diamond', 's', 'propriety', 'attitude', 'to', 'it', 's', 'hardware', 'it', 's', 'impossible', 'to', 'get', 'free', 'information', 'from', 'them', 'about', 'their', 'chips', 'specifically', 'their', 'dot', 'clocks', 'without', 'paying', 'and', 'signing', 'non', 'disclosure', 'agreements', 'this', 'made', 'it', 'impossible', 'for', 'the', 'free', 'software', 'foundation', 'to', 'provide', 'x', 'windows', 'compatibility', 'with', 'these', 'cards', 'as', 'diamond

### 4. Construcción del Modelo de Unigramas

En esta sección, construimos el modelo de unigramas para ambos datasets. Un unigrama es simplemente una palabra en sí misma. Contamos la frecuencia de cada palabra y calculamos su probabilidad aplicando el suavizado de Laplace, que asegura que todas las palabras, incluso aquellas que no aparecieron en el conjunto de entrenamiento, tengan una probabilidad mayor a 0.


In [5]:
def construirModeloUnigrama(oracionesSeparadas):
    """
    Construye el modelo de unigramas a partir de las oraciones tokenizadas

    Parámetros:
    oracionesSeparadas (list): Lista de oraciones tokenizadas

    Retorna:
    tuple: Un Counter con las frecuencias de los unigramas y el total de palabras
    """
    cuentaUnigrama = Counter()
    totalPalabras = 0
    for oracion in oracionesSeparadas:
        for palabra in oracion:
            cuentaUnigrama[palabra] += 1
            totalPalabras += 1
    return cuentaUnigrama, totalPalabras

unigrama20N, totalPalabras20N = construirModeloUnigrama(oracionesSeparadas20N)
unigramaBAC, totalPalabrasBAC = construirModeloUnigrama(oracionesSeparadasBAC)

# Tamaños de vocabulario
tamanoVocabulario20N = len(unigrama20N)
tamanoVocabularioBAC = len(unigramaBAC)

print(f"Tamaño del vocabulario 20N: {tamanoVocabulario20N}")
print(f"Tamaño del vocabulario BAC: {tamanoVocabularioBAC}")


Tamaño del vocabulario 20N: 71294
Tamaño del vocabulario BAC: 328708


In [6]:
def probabilidadUnigrama(cuentaUnigrama, totalPalabras, palabra, tamanoVocabulario):
    """
    Calcula la probabilidad de un unigrama

    Parámetros:
    cuentaUnigrama (Counter): Contador de frecuencias de unigramas
    totalPalabras (int): Total de palabras en el corpus
    palabra (str): Palabra para la cual se calcula la probabilidad
    tamanoVocabulario (int): Tamaño del vocabulario

    Retorna:
    float: Probabilidad suavizada del unigrama
    """
    return (cuentaUnigrama.get(palabra, 0) + 1) / (totalPalabras + tamanoVocabulario)

palabra = "NUM"
print(f"Probabilidad de '{palabra}' en 20N: {probabilidadUnigrama(unigrama20N, totalPalabras20N, palabra, tamanoVocabulario20N)}")
print(f"Probabilidad de '{palabra}' en BAC: {probabilidadUnigrama(unigramaBAC, totalPalabrasBAC, palabra, tamanoVocabularioBAC)}")


Probabilidad de 'NUM' en 20N: 0.049626061618955615
Probabilidad de 'NUM' en BAC: 0.016980981559552168


### 5. Construcción del Modelo de Bigramas

Ahora creamos el modelo de bigramas, que calcula las probabilidades de aparición de una palabra dada la palabra anterior en la secuencia.

In [7]:
def construirModeloBigrama(oracionesSeparadas):
    """
    Construye el modelo de bigramas a partir de las oraciones tokenizadas

    Parámetros:
    oracionesSeparadas (list): Lista de oraciones tokenizadas

    Retorna:
    tuple: Un defaultdict de Counters con las frecuencias de bigramas y el total de bigramas
    """
    cuentaBigramas = defaultdict(Counter)
    totalBigramas = 0
    for oracion in oracionesSeparadas:
        for i in range(len(oracion) - 1):
            palabra1 = oracion[i]
            palabra2 = oracion[i+1]
            cuentaBigramas[palabra1][palabra2] += 1
            totalBigramas += 1
    return cuentaBigramas, totalBigramas

bigrama20N, totalBigramas20N = construirModeloBigrama(oracionesSeparadas20N)
bigramaBAC, totalBigramasBAC = construirModeloBigrama(oracionesSeparadasBAC)

print(f"Total de bigramas en 20N: {totalBigramas20N}")
print(f"Total de bigramas en BAC: {totalBigramasBAC}")


Total de bigramas en 20N: 4380316
Total de bigramas en BAC: 111110021


In [8]:
def probabilidadBigrama(cuentaBigramas, cuentaUnigrama, palabra1, palabra2, tamanoVocabulario):
    """
    Calcula la probabilidad de un bigrama

    Parámetros:
    cuentaBigramas (defaultdict(Counter)): Frecuencias de bigramas
    cuentaUnigrama (Counter): Frecuencias de unigramas
    palabra1 (str): Primera palabra del bigrama
    palabra2 (str): Segunda palabra del bigrama
    tamanoVocabulario (int): Tamaño del vocabulario

    Retorna:
    float: Probabilidad suavizada del bigrama
    """
    frecuenciaBigram = cuentaBigramas.get(palabra1, {}).get(palabra2, 0)
    frecuenciaUnigram = cuentaUnigrama.get(palabra1, 0)
    return (frecuenciaBigram + 1) / (frecuenciaUnigram + tamanoVocabulario)

palabra1 = "NUM"
palabra2 = "NUM"
print(f"Probabilidad de '{palabra1} {palabra2}' en 20N: {probabilidadBigrama(bigrama20N, unigrama20N, palabra1, palabra2, tamanoVocabulario20N)}")
print(f"Probabilidad de '{palabra1} {palabra2}' en BAC: {probabilidadBigrama(bigramaBAC, unigramaBAC, palabra1, palabra2, tamanoVocabularioBAC)}")


Probabilidad de 'NUM NUM' en 20N: 0.2827226024474749
Probabilidad de 'NUM NUM' en BAC: 0.1300816697730177


### 6. Construcción del Modelo de Trigramas

Por último, construimos el modelo de trigramas, que calcula las probabilidades de que una palabra aparezca en función de las dos palabras anteriores. Al igual que con los otros modelos, utilizamos suavizado de Laplace para evitar que la probabilidad sea cero si un trigramq no está



In [9]:
def crearDefaultdictCounter():
    """
    Crea un defaultdict de Counter, para utilizar en el modelo de trigramas

    Retorna:
    defaultdict(Counter): Un defaultdict que retorna un Counter por defecto
    """
    return defaultdict(Counter)

def construirModeloTrigrama(oracionesSeparadas):
    """
    Construye el modelo de trigramas a partir de las oraciones tokenizadas

    Parámetros:
    oracionesSeparadas (list): Lista de oraciones tokenizadas

    Retorna:
    tuple: Un defaultdict anidado con las frecuencias de trigramas y el total de trigramas
    """
    cuentaTrigramas = defaultdict(crearDefaultdictCounter)  
    totalTrigramas = 0
    for oracion in oracionesSeparadas:
        for i in range(len(oracion) - 2):
            palabra1 = oracion[i]
            palabra2 = oracion[i+1]
            palabra3 = oracion[i+2]
            cuentaTrigramas[palabra1][palabra2][palabra3] += 1
            totalTrigramas += 1
    return cuentaTrigramas, totalTrigramas

trigrama20N, totalTrigramas20N = construirModeloTrigrama(oracionesSeparadas20N)
trigramaBAC, totalTrigramasBAC = construirModeloTrigrama(oracionesSeparadasBAC)

print(f"Total de trigramas en 20N: {totalTrigramas20N}")
print(f"Total de trigramas en BAC: {totalTrigramasBAC}")


Total de trigramas en 20N: 4365281
Total de trigramas en BAC: 110844901


In [10]:
def probabilidadTrigrama(cuentaTrigramas, cuentaBigramas, palabra1, palabra2, palabra3, tamanoVocabulario):
    """
    Calcula la probabilidad de un trigrama

    Parámetros:
    cuentaTrigramas (defaultdict): Frecuencias de trigramas
    cuentaBigramas (defaultdict(Counter)): Frecuencias de bigramas
    palabra1 (str): Primera palabra del trigrama
    palabra2 (str): Segunda palabra del trigrama
    palabra3 (str): Tercera palabra del trigrama
    tamanoVocabulario (int): Tamaño del vocabulario

    Retorna:
    float: Probabilidad suavizada del trigrama
    """
    frecuenciaTrigram = cuentaTrigramas.get(palabra1, {}).get(palabra2, {}).get(palabra3, 0)
    frecuenciaBigram = cuentaBigramas.get(palabra1, {}).get(palabra2, 0)
    return (frecuenciaTrigram + 1) / (frecuenciaBigram + tamanoVocabulario)

palabra1 = "NUM"
palabra2 = "NUM"
palabra3 = "NUM"
print(f"Probabilidad de '{palabra1} {palabra2} {palabra3}' en 20N: {probabilidadTrigrama(trigrama20N, bigrama20N, palabra1, palabra2, palabra3, tamanoVocabulario20N)}")
print(f"Probabilidad de '{palabra1} {palabra2} {palabra3}' en BAC: {probabilidadTrigrama(trigramaBAC, bigramaBAC, palabra1, palabra2, palabra3, tamanoVocabularioBAC)}")


Probabilidad de 'NUM NUM NUM' en 20N: 0.2762623444373791
Probabilidad de 'NUM NUM NUM' en BAC: 0.12507562155254687


### 7. Guardar los Modelos


In [11]:
def guardarModelo(modelo, nombreArchivo):
    """
    Guarda un modelo utilizando pickle

    Parámetros:
    modelo: Modelo a guardar
    nombreArchivo (str): Nombre del archivo donde se guardará el modelo.
    """
    with open(nombreArchivo, 'wb') as archivo:
        pickle.dump(modelo, archivo)

# Guardar los modelos de 20N
guardarModelo(unigrama20N, '20N_01_unigramas.pkl')
guardarModelo(bigrama20N, '20N_01_bigramas.pkl')
guardarModelo(trigrama20N, '20N_01_trigramas.pkl')

# Guardar los modelos de BAC
guardarModelo(unigramaBAC, 'BAC_01_unigramas.pkl')
guardarModelo(bigramaBAC, 'BAC_01_bigramas.pkl')
guardarModelo(trigramaBAC, 'BAC_01_trigramas.pkl')

print("Modelos guardados exitosamente.")


Modelos guardados exitosamente.
